In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from pathlib import Path
import re
from collections import Counter
import pickle
from tqdm import tqdm
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, Input, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Set the paths
data_dir = Path('data/McGill-Billboard')
chordino_dir = data_dir / 'chordino'
lab_dir = data_dir / 'lab'
annotations_dir = data_dir / 'annotations'
index_path = index_path = data_dir / 'index.csv'


In [ ]:
# Load the dataset index
index_df = pd.read_csv(index_path)
print(f"Total entries in index: {len(index_df)}")
print(f"Entries with complete data: {index_df['title'].notna().sum()}")


In [ ]:
# Drop entries with missing incomplete/unavailable data
index_df = index_df.dropna(subset=['title'])

# Verify new dataframe
print(f'Total Entries after cleaning: {len(index_df)}')

In [ ]:
index_df.head()

In [ ]:
# Display basic information
print(f"Number of unique songs: {index_df['title'].nunique()}")
print(f"Number of unique artists: {index_df['artist'].nunique()}")

In [ ]:
# Convert chart_date to datetime
index_df['chart_date'] = pd.to_datetime(index_df['chart_date'])

# Extract year and decade
index_df['year'] = index_df['chart_date'].dt.year
index_df['decade'] = (index_df['year'] // 10) * 10

In [ ]:
decade_counts = index_df['decade'].value_counts().sort_index()

print(decade_counts)

In [ ]:
# Check available directories
song_ids = [d.name for d in chordino_dir.iterdir() if d.is_dir()]
print(f"Number of songs with chroma features: {len(song_ids)}")

# Check if the same songs have lab files
lab_ids = [d.name for d in lab_dir.iterdir() if d.is_dir()]
print(f"Number of songs with lab files: {len(lab_ids)}")

# Find songs that have both chroma and labels
common_ids = set(song_ids).intersection(set(lab_ids))
print(f"Number of songs with both features and labels: {len(common_ids)}")

In [ ]:
# Explore one example
example_id = list(common_ids)[random.randint(0, len(common_ids))]

# Load chroma features for the example song
chroma_path = chordino_dir / example_id / 'bothchroma.csv'
tuning_path = chordino_dir / example_id / 'tuning.csv'

if chroma_path.exists() and tuning_path.exists():
    chroma_data = pd.read_csv(chroma_path, header=None)
    tuning_data = pd.read_csv(tuning_path, header=None)
    
    print(f"\nChroma shape for song {example_id}: {chroma_data.shape}")
    print(f"Tuning shape for song {example_id}: {tuning_data.shape}")
    
    # Display a sample of the chroma data
    print("\nSample of chroma data:")
    print(chroma_data.head())
else:
    print(f"Chroma or tuning data not found for song {example_id}")

# Load chord labels for the example song
lab_path = lab_dir / example_id / "full.lab"

if lab_path.exists():
    # Read the lab file (tab-separated with no header)
    lab_data = pd.read_csv(lab_path, sep='\t', header=None, names=['start_time', 'end_time', 'chord'])
    
    print(f"\nNumber of chord segments for song {example_id}: {len(lab_data)}")
    print("\nSample of chord labels:")
    print(lab_data.head())
    
    # Count the unique chords in this song
    print(f"\nNumber of unique chords in song {example_id}: {lab_data['chord'].nunique()}")
    print("\nMost common chords:")
    print(lab_data['chord'].value_counts().head(10))
else:
    print(f"Lab file not found for song {example_id}")

In [ ]:
print(f"Chroma shape for song {example_id}: {chroma_data.shape}")
print("\nSample of chroma data:")
print(chroma_data.head())

# Plot chroma data
plt.figure(figsize=(12, 4))
sns.heatmap(chroma_data.iloc[:, 2:].T, cmap='coolwarm', cbar=True)
plt.title(f'Chroma Feature Heatmap for Song {example_id}')
plt.xlabel('Time frames')
plt.ylabel('Pitch Class')
plt.show()

In [ ]:
# Load chord labels
lab_data = pd.read_csv(lab_dir / example_id / "full.lab", sep='\t', names=['start', 'end', 'chord'])

print(f"Number of chord segments for song {example_id}: {len(lab_data)}")
print("\nSample of chord segments:")
print(lab_data.head())

In [ ]:
# Count unique chords
unique_chords = lab_data['chord'].nunique()
print(f"Number of unique chords in song {example_id}: {unique_chords}")

In [ ]:
# Most frequent chords
most_common_chords = lab_data['chord'].value_counts().head(10)
print("Most common chords:")
print(most_common_chords)

In [ ]:
def load_chroma(song_id):
    """Load chroma features for a given song ID"""
    chroma_path = chordino_dir / song_id / 'bothchroma.csv'
    
    if not chroma_path.exists():
        return None
    
    # Read the file as text first to understand the structure
    with open(chroma_path, 'r') as f:
        first_line = f.readline().strip()
    
    # Check the structure of the file
    columns = first_line.split(',')
    
    # Based on your description, we need to handle two different formats
    if len(columns) >= 14:  # First two columns are non-numeric, followed by 12 chroma values
        # Load the chroma features, skipping the non-numeric metadata columns
        chroma_data = pd.read_csv(chroma_path, header=None, usecols=range(2, 14))
    else:
        # If the format is different, try to determine which columns contain the chroma features
        # For now, we'll just use all columns and convert non-numeric to NaN
        chroma_data = pd.read_csv(chroma_path, header=None)
        # Convert to numeric, coercing errors to NaN
        for col in chroma_data.columns:
            chroma_data[col] = pd.to_numeric(chroma_data[col], errors='coerce')
        # Drop columns with NaN values (these are likely non-numeric metadata columns)
        chroma_data = chroma_data.dropna(axis=1)
    
    # Convert to numpy array, ensuring all values are numeric
    chroma_array = chroma_data.values.astype(np.float32)
    
    # Check for invalid values and replace with zeros
    chroma_array = np.nan_to_num(chroma_array)
    
    return chroma_array

def load_chord_labels(song_id):
    """Load chord labels for a given song ID"""
    lab_path = lab_dir / song_id / "full.lab"
    
    if not lab_path.exists():
        return None
    
    # Load the chord labels
    lab_data = pd.read_csv(lab_path, sep='\t', header=None, names=['start_time', 'end_time', 'chord'])
    
    return lab_data

def get_chroma_timestamps(chroma_array, hop_size=0.01):
    """Generate timestamps for chroma features"""
    # Chordino typically uses a 0.01s hop size (10ms)
    return np.arange(len(chroma_array)) * hop_size

def align_chroma_with_chords(chroma_array, chord_data, chroma_hop_size=0.01):
    """Align chroma features with chord labels"""
    chroma_times = get_chroma_timestamps(chroma_array, chroma_hop_size)
    aligned_chords = []
    
    for i, time in enumerate(chroma_times):
        # Find the chord label for this time point
        chord_idx = ((chord_data['start_time'] <= time) & (chord_data['end_time'] > time)).idxmax()
        if pd.isna(chord_idx):  # No matching chord found
            chord = "N"  # No chord (silence or undefined)
        else:
            chord = chord_data.loc[chord_idx, 'chord']
        
        aligned_chords.append(chord)
    
    return aligned_chords

In [ ]:
def build_dataset(song_ids, max_songs=100):
    """Build a dataset of chroma features and chord labels"""
    X = []  # Chroma features
    y = []  # Chord labels
    
    processed_songs = 0
    all_chords = []
    failed_songs = 0
    
    for song_id in tqdm(song_ids):
        if processed_songs >= max_songs:
            break
            
        try:
            # Load chroma and chord data
            chroma_array = load_chroma(song_id)
            chord_data = load_chord_labels(song_id)
            
            if chroma_array is None or chord_data is None:
                continue
                
            # Make sure chroma data is strictly numerical and contains no NaNs or Infs
            if not np.isfinite(chroma_array).all():
                print(f"Warning: Song {song_id} has non-finite values in chroma. Skipping.")
                failed_songs += 1
                continue
                
            # Align chroma with chords
            aligned_chords = align_chroma_with_chords(chroma_array, chord_data)
            
            # Add to dataset
            X.append(chroma_array)
            y.append(aligned_chords)
            
            # Track all unique chords
            all_chords.extend(set(aligned_chords))
            
            processed_songs += 1
        except Exception as e:
            print(f"Error processing song {song_id}: {e}")
            failed_songs += 1
    
    print(f"Successfully processed {processed_songs} songs, failed to process {failed_songs} songs")
    return X, y, list(set(all_chords))

# Let's start with a smaller dataset
sample_size = min(100, len(common_ids))
song_subset = list(common_ids)[:sample_size]

# Build the initial dataset
X_songs, y_songs, unique_chords = build_dataset(song_subset, max_songs=sample_size)

print(f"Processed {len(X_songs)} songs")
print(f"Number of unique chords: {len(unique_chords)}")
print(f"Some example chords: {unique_chords[:10]}")

In [ ]:
# Flatten the dataset to frame-level
def flatten_dataset(X_songs, y_songs):
    """Convert song-level dataset to frame-level"""
    X_frames = []
    y_frames = []
    
    for i in range(len(X_songs)):
        X_frames.extend(X_songs[i])
        y_frames.extend(y_songs[i])
        
    # Convert to numpy array and ensure correct data type
    X_frames_np = np.array(X_frames, dtype=np.float32)
    y_frames_np = np.array(y_frames)
    
    return X_frames_np, y_frames_np

In [ ]:
# Check if we have data to process
if len(X_songs) > 0:
    X_flat, y_flat = flatten_dataset(X_songs, y_songs)

    print(f"Total frames: {len(X_flat)}")
    print(f"Feature shape: {X_flat.shape}")
    
    # Check for any remaining non-finite values
    non_finite_count = np.sum(~np.isfinite(X_flat))
    if non_finite_count > 0:
        print(f"Warning: {non_finite_count} non-finite values found in the flattened data.")
        # Replace non-finite values with zeros
        X_flat = np.nan_to_num(X_flat)
        
    # Make sure the data is normalized (important for neural networks)
    # Chroma features are typically already between 0 and 1, but let's check
    max_value = np.max(X_flat)
    if max_value > 1.0:
        print(f"Max chroma value is {max_value}, normalizing to [0, 1] range")
        X_flat = X_flat / max_value

    # Encode the chord labels
    label_encoder = LabelEncoder()
    try:
        y_encoded = label_encoder.fit_transform(y_flat)
        
        # Save the label encoder for later use
        os.makedirs('models', exist_ok=True)
        with open('models/chord_label_encoder.pkl', 'wb') as f:
            pickle.dump(label_encoder, f)
        
        # Get the number of classes
        n_classes = len(label_encoder.classes_)
        print(f"Number of chord classes: {n_classes}")
        
        # Convert to one-hot encoding
        y_onehot = to_categorical(y_encoded, num_classes=n_classes)
        
        # Split into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_flat, y_onehot, test_size=0.2, random_state=42)
        
        print(f"Training set: {X_train.shape}")
        print(f"Testing set: {X_test.shape}")
    except Exception as e:
        print(f"Error in encoding labels: {e}")
        print("This may be due to empty or invalid chord labels.")
else:
    print("No data to process. Check the dataset loading steps above for errors.")

In [ ]:
# Count chord occurrences
chord_counts = Counter(y_flat)

# Get the top 20 chords
top_chords = chord_counts.most_common(20)
print("Top 20 chords:")
for chord, count in top_chords:
    print(f"{chord}: {count}")

# Visualize chord distribution
plt.figure(figsize=(12, 6))
counts = [count for _, count in top_chords]
labels = [chord for chord, _ in top_chords]

plt.bar(range(len(counts)), counts)
plt.xticks(range(len(counts)), labels, rotation=45, ha='right')
plt.title('Top 20 Chord Frequencies')
plt.tight_layout()
plt.show()

In [ ]:
# Build and Train Models

def create_feedforward_model(input_shape, n_classes):
    """Create a simple feedforward neural network optimized for chroma input"""
    model = Sequential([
        # Input layer - shape is (12,) for chroma vectors
        Dense(128, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.3),
        
        # Hidden layers
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        # Output layer
        Dense(n_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def create_cnn_model(input_shape, n_classes):
    """Create a 1D CNN model for chroma processing"""
    # For chroma vectors, we reshape to (12, 1) to apply 1D convolutions
    model = Sequential([
        # Input layer
        Input(shape=input_shape),
        
        # Reshape to have a time dimension (treating the 12 chroma bins as a sequence)
        tf.keras.layers.Reshape((input_shape[0], 1)),
        
        # First conv block
        Conv1D(64, kernel_size=3, padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        # Second conv block
        Conv1D(128, kernel_size=3, padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        # Flatten and dense layers
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        # Output layer
        Dense(n_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def create_lstm_model(input_shape, n_classes):
    """Create an LSTM model for chroma sequences"""
    # Reshape chroma vectors to sequences
    model = Sequential([
        # Input layer
        Input(shape=input_shape),
        
        # Reshape to have a time dimension
        tf.keras.layers.Reshape((input_shape[0], 1)),
        
        # LSTM layers
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.4),
        
        LSTM(64),
        BatchNormalization(),
        Dropout(0.3),
        
        # Dense layers
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        # Output layer
        Dense(n_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Get input shape from training data
input_shape = (X_train.shape[1],)
print(f"Input shape for models: {input_shape}")

# Define callbacks for training
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-5, verbose=1),
    ModelCheckpoint('models/best_model_checkpoint.h5', save_best_only=True, verbose=1)
]

# Train the feedforward model
print("\n--- Training Feedforward Model ---")

ff_model = create_feedforward_model(input_shape, n_classes)
print(ff_model.summary())

In [ ]:
history_ff = ff_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Save the model
ff_model.save('models/chord_classifier_feedforward.keras')
print("Feedforward model saved successfully")

In [ ]:
# Train the CNN model

cnn_model = create_cnn_model(input_shape, n_classes)
print(cnn_model.summary())

history_cnn = cnn_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Save the model
cnn_model.save('models/chord_classifier_cnn.h5')
print("CNN model saved successfully")

In [ ]:
# Train the LSTM model

lstm_model = create_lstm_model(input_shape, n_classes)
print(lstm_model.summary())

history_lstm = lstm_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Save the model
lstm_model.save('models/chord_classifier_lstm.h5')
print("LSTM model saved successfully")

In [ ]:
print("\n--- Model Evaluation ---")
    
# Function to safely evaluate models
def safe_evaluate(model, model_name):
    try:
        scores = model.evaluate(X_test, y_test, verbose=1)
        print(f"{model_name} - Test Loss: {scores[0]:.4f}, Test Accuracy: {scores[1]:.4f}")
        return scores
    except Exception as e:
        print(f"Error evaluating {model_name}: {e}")
        return None

# Evaluate each model if it exists
if 'ff_model' in locals():
    ff_scores = safe_evaluate(ff_model, "Feedforward Model")

if 'cnn_model' in locals():
    cnn_scores = safe_evaluate(cnn_model, "CNN Model")

if 'lstm_model' in locals():
    lstm_scores = safe_evaluate(lstm_model, "LSTM Model")

In [ ]:
# Plot training history
def plot_history(history, title):
    if history is None:
        print(f"No training history available for {title}")
        return
        
    try:
        plt.figure(figsize=(12, 4))
        
        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title(f'{title} - Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='lower right')
        
        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title(f'{title} - Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper right')
        
        plt.tight_layout()
        plt.savefig(f"models/{title.replace(' ', '_').lower()}_training_history.png")
        plt.show()
    except Exception as e:
        print(f"Error plotting history for {title}: {e}")

In [ ]:
# Plot histories if available
if 'history_ff' in locals():
    plot_history(history_ff, 'Feedforward Model')

if 'history_cnn' in locals():
    plot_history(history_cnn, 'CNN Model')

if 'history_lstm' in locals():
    plot_history(history_lstm, 'LSTM Model')

In [ ]:
# Function to get confusion matrix and classification report
def model_analysis(model, X, y, label_encoder, title):
    """Analyze model performance with confusion matrix and classification report"""
    # Get predictions
    y_pred_proba = model.predict(X)
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_true = np.argmax(y, axis=1)
    
    # Get class names
    class_names = label_encoder.classes_
    
    # Create confusion matrix (limit to top N classes for readability)
    top_n = 20
    top_classes_idx = np.bincount(y_true).argsort()[-top_n:]
    
    # Filter to only include top classes
    mask_true = np.isin(y_true, top_classes_idx)
    y_true_filtered = y_true[mask_true]
    y_pred_filtered = y_pred[mask_true]
    
    # Confusion matrix
    cm = confusion_matrix(y_true_filtered, y_pred_filtered)
    
    # Plot confusion matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=[class_names[i] for i in top_classes_idx],
        yticklabels=[class_names[i] for i in top_classes_idx]
    )
    plt.title(f'{title} - Confusion Matrix (Top {top_n} Classes)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Classification report
    print(f"\n{title} - Classification Report:")
    print(classification_report(
        y_true, 
        y_pred, 
        target_names=class_names,
        labels=range(len(class_names)),
        zero_division=0
    ))

In [ ]:
# Analyze the best performing model
best_model = lstm_model  # Change this based on your results
model_analysis(best_model, X_test, y_test, label_encoder, 'Best Model')